# TABELAS OURO


## Vamos começar criando as dimensões e depois os fatos

### dimensão Usuário (gold.dim_usuario)

_### Criando um schema separado para as tabelas gold do modelo estrela, separado das tabelas silver e bronze que estão no schema default_

In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA gold;

### Vamos começar criando as dimensões e depois os fatos

In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA gold;

CREATE OR REPLACE TABLE dim_usuario AS
SELECT
  ROW_NUMBER() OVER (ORDER BY user_id) AS usuario_id,   -- surrogate key sequencial
  nome,
  email,
  assinatura_usuario AS plano,
  telefone,
  creation_dt AS data_criacao,
  escritorio AS escritorio_nome
FROM workspace.default.silver_users;


In [0]:
%sql
DESCRIBE TABLE gold.dim_usuario;
    


In [0]:
%sql
SELECT * FROM gold.dim_usuario 
WHERE telefone IS NOT NULL
limit 10;

In [0]:
%sql
CREATE OR REPLACE TABLE dim_projeto AS
SELECT
  -- surrogate key sequencial
  ROW_NUMBER() OVER (ORDER BY project_id) AS projeto_id,

  -- atributos da dimensão
  nome_projeto,
  escritorio,
  equipe_projeto,
  creation_dt AS data_criacao,
  metragem,
  qnt_visita

FROM workspace.default.silver_projetos;

In [0]:
%sql
select * from dim_projeto limit 10;

In [0]:
%sql
CREATE OR REPLACE TABLE dim_visita AS
SELECT
    -- surrogate key sequencial
    ROW_NUMBER() OVER (ORDER BY visita_uuid) AS visita_id,

    -- identificador natural
    visita_uuid,

    -- atributos da dimensão
    nome AS nome_visita,
    observacao AS anotacoes

FROM workspace.default.silver_visitas;


In [0]:
%sql
select * from dim_visita limit 20;
    

In [0]:
%sql
CREATE OR REPLACE TABLE dim_pendencia AS
SELECT
    -- surrogate key sequencial
    ROW_NUMBER() OVER (ORDER BY pendencia_uuid) AS pendencia_id,

    -- identificador natural da pendência
    pendencia_uuid,

    -- texto associado à pendência
    titulo AS texto_pendencia

FROM workspace.default.silver_pendencias;


In [0]:
%sql
SELECT * FROM dim_pendencia ORDER BY pendencia_id LIMIT 20;


In [0]:
%sql
CREATE OR REPLACE TABLE dim_tempo AS
WITH calendario AS (
    SELECT sequence(
        to_date('2023-01-01'),
        to_date('2028-12-31'),
        interval 1 day
    ) AS datas
),
datas_explodidas AS (
    SELECT explode(datas) AS data
    FROM calendario
)
SELECT
    ROW_NUMBER() OVER (ORDER BY data) AS tempo_id,
    data,
    day(data) AS dia,
    month(data) AS mes,
    year(data) AS ano,
    quarter(data) AS trimestre,
    date_format(data, 'EEEE') AS dia_semana
FROM datas_explodidas;


In [0]:
%sql
SELECT * FROM dim_tempo ORDER BY tempo_id LIMIT 10;


In [0]:
%sql
show tables in default;

In [0]:
%sql
CREATE OR REPLACE TABLE gold.dim_visita AS SELECT * FROM default.dim_visita;
DROP TABLE default.dim_visita;

CREATE OR REPLACE TABLE gold.dim_pendencia AS SELECT * FROM default.dim_pendencia;
DROP TABLE default.dim_pendencia;

CREATE OR REPLACE TABLE gold.dim_tempo AS SELECT * FROM default.dim_tempo;
DROP TABLE default.dim_tempo;


In [0]:
%sql
create schema if not exists gold;

In [0]:
%sql
show tables in gold;

In [0]:
%sql
show tables in default;

In [0]:
%sql
show tables in gold;

## # Criando agora os fatos

In [0]:
%sql
CREATE OR REPLACE TABLE gold.fato_visita_criada AS
WITH base AS (
    SELECT
        v.*,

        size(
            filter(
                split(fotos, ','),
                x -> trim(x) <> ''
            )
        ) AS qtd_fotos,

        CASE 
            WHEN v.modified_dt <> v.creation_dt THEN true
            ELSE false
        END AS foi_modificada
    FROM workspace.default.silver_visitas v
),

pendencias_por_visita AS (
    SELECT
        visita_pendencias AS nome_visita,
        COUNT(*) AS qtd_pendencias
    FROM workspace.default.silver_pendencias
    GROUP BY visita_pendencias
)

SELECT
    ROW_NUMBER() OVER (ORDER BY v.creation_dt, v.nome) AS id_fato_visita,

    p.projeto_id,
    u.usuario_id,
    t.tempo_id,

    v.nome AS nome_visita,
    v.escritorio,

    v.qtd_fotos,
    COALESCE(pp.qtd_pendencias, 0) AS qtd_pendencias,
    v.foi_modificada,

    v.creation_dt AS timestamp_vis,
    v.modified_dt AS timestamp_vis_mod

FROM base v
LEFT JOIN gold.dim_projeto p
       ON v.projeto_visita = p.nome_projeto
LEFT JOIN gold.dim_usuario u
       ON v.creator = u.email
LEFT JOIN gold.dim_tempo t
       ON v.creation_dt = t.data
LEFT JOIN pendencias_por_visita pp
       ON v.nome = pp.nome_visita;


In [0]:
%sql
select * from fato_visita_criada limit 15;

In [0]:
%sql
SELECT COUNT(*) AS visitas_sem_projeto
FROM gold.fato_visita_criada
WHERE projeto_id IS NULL;


In [0]:
%sql
SELECT COUNT(*) AS visitas_sem_usuario
FROM gold.fato_visita_criada
WHERE usuario_id IS NULL;


In [0]:
%sql
SELECT COUNT(*) AS visitas_sem_tempo
FROM gold.fato_visita_criada
WHERE tempo_id IS NULL;


In [0]:
%sql
SELECT nome_visita, qtd_pendencias
FROM gold.fato_visita_criada
ORDER BY qtd_pendencias DESC
LIMIT 20;


In [0]:
%sql
SELECT nome_visita, qtd_fotos, timestamp_vis
FROM gold.fato_visita_criada
ORDER BY qtd_fotos DESC
LIMIT 20;


In [0]:
%sql
SELECT *
FROM gold.fato_visita_criada
ORDER BY id_fato_visita
LIMIT 20;


In [0]:
%sql
SELECT f.nome_visita, s.projeto_visita
FROM gold.fato_visita_criada f
LEFT JOIN workspace.default.silver_visitas s
       ON f.nome_visita = s.nome
WHERE f.projeto_id IS NULL;


In [0]:
%sql
SELECT f.nome_visita, s.creator
FROM gold.fato_visita_criada f
LEFT JOIN workspace.default.silver_visitas s
       ON f.nome_visita = s.nome
WHERE f.usuario_id IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE gold.fato_visita_criada AS
WITH base AS (
    SELECT
        v.*,

        -- Contagem de fotos
        size(
            filter(
                split(fotos, ','),
                x -> trim(x) <> ''
            )
        ) AS qtd_fotos,

        -- Flag de modificação
        CASE 
            WHEN v.modified_dt <> v.creation_dt THEN true
            ELSE false
        END AS foi_modificada
    FROM workspace.default.silver_visitas v

    -- FILTROS DE QUALIDADE (IMPORTANTE)
    WHERE v.nome IS NOT NULL            -- protege integridade
      AND v.creator IS NOT NULL         -- protege FK usuario
      AND v.projeto_visita IS NOT NULL  -- protege FK projeto
),
pendencias_por_visita AS (
    SELECT
        visita_pendencias AS nome_visita,
        COUNT(*) AS qtd_pendencias
    FROM workspace.default.silver_pendencias
    GROUP BY visita_pendencias
)

SELECT
    ROW_NUMBER() OVER (ORDER BY v.creation_dt, v.nome) AS id_fato_visita,

    p.projeto_id,
    u.usuario_id,
    t.tempo_id,

    v.nome AS nome_visita,
    v.escritorio,

    v.qtd_fotos,
    COALESCE(pp.qtd_pendencias, 0) AS qtd_pendencias,
    v.foi_modificada,

    v.creation_dt AS timestamp_vis,
    v.modified_dt AS timestamp_vis_mod

FROM base v
LEFT JOIN gold.dim_projeto p
       ON v.projeto_visita = p.nome_projeto
LEFT JOIN gold.dim_usuario u
       ON v.creator = u.email
LEFT JOIN gold.dim_tempo t
       ON v.creation_dt = t.data
LEFT JOIN pendencias_por_visita pp
       ON v.nome = pp.nome_visita;


In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_visita_criada WHERE projeto_id IS NULL;

In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_visita_criada WHERE usuario_id IS NULL;


In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_visita_criada WHERE tempo_id IS NULL;


In [0]:
%sql
CREATE OR REPLACE TABLE gold.fato_pendencia_criada AS
WITH base AS (
    SELECT
        p.*,

        -- Flags booleanas (já são boolean na silver, mas garantimos o tipo)
        CAST(p.urgente AS BOOLEAN)           AS eh_urgente,
        CAST(p.resolvida AS BOOLEAN)         AS resolvida_flag,
        CAST(p.envolver_cliente AS BOOLEAN)  AS envolver_cliente_flag

    FROM workspace.default.silver_pendencias p

    -- 🔒 FILTROS DE QUALIDADE
    WHERE p.creator IS NOT NULL
      AND p.creator <> ''
      AND p.projeto_pendencias IS NOT NULL
      AND p.projeto_pendencias <> ''
      AND p.visita_pendencias IS NOT NULL
      AND p.visita_pendencias <> ''
      AND p.titulo IS NOT NULL
      AND p.titulo <> ''
)

SELECT
    -- surrogate key
    ROW_NUMBER() OVER (ORDER BY b.creation_dt, b.titulo) AS id_fato_pendencia,

    -- FKs
    u.usuario_id,
    pr.projeto_id,
    v.visita_id,
    t.tempo_id,
    d.pendencia_id        AS pendencia_desc_id,

    -- métricas / atributos de status
    b.eh_urgente,
    b.resolvida_flag      AS resolvida,
    b.envolver_cliente_flag AS envolver_cliente,

    -- datas
    b.creation_dt         AS date_pendencia,
    b.modified_dt         AS date_pendencia_mod

FROM base b

-- FK projeto
LEFT JOIN gold.dim_projeto pr
       ON b.projeto_pendencias = pr.nome_projeto

-- FK visita
LEFT JOIN gold.dim_visita v
       ON b.visita_pendencias = v.nome_visita

-- FK usuário
LEFT JOIN gold.dim_usuario u
       ON b.creator = u.email

-- FK tempo
LEFT JOIN gold.dim_tempo t
       ON b.creation_dt = t.data

-- FK descrição da pendência (usando o UUID, mais robusto)
LEFT JOIN gold.dim_pendencia d
       ON b.pendencia_uuid = d.pendencia_uuid;


In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_pendencia_criada WHERE projeto_id IS NULL;

In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_foto_adicionada WHERE projeto_id IS NULL;

In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_foto_adicionada WHERE usuario_id IS NULL;

In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_foto_adicionada WHERE visita_id IS NULL;

In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_foto_adicionada WHERE tempo_id IS NULL;

In [0]:
%sql
select * from gold.fato_foto_adicionada limit 15;

In [0]:
%sql
SELECT COUNT(*) FROM workspace.default.silver_fotos;


In [0]:
%sql
SELECT COUNT(*) AS fotos_com_creator
FROM workspace.default.silver_fotos
WHERE creator IS NOT NULL AND trim(creator) <> '';


In [0]:
%sql
SELECT COUNT(*) AS fotos_com_projeto
FROM workspace.default.silver_fotos
WHERE projeto IS NOT NULL AND trim(projeto) <> '';


In [0]:
%sql
SELECT COUNT(*) AS fotos_com_visita
FROM workspace.default.silver_fotos
WHERE visita IS NOT NULL AND trim(visita) <> '';


In [0]:
%sql
SELECT COUNT(*) AS fotos_com_imagem
FROM workspace.default.silver_fotos
WHERE imagem_url IS NOT NULL AND trim(imagem_url) <> '';


In [0]:
%sql
CREATE OR REPLACE TABLE gold.fato_foto_adicionada AS
WITH base AS (
    SELECT
        f.*,

        -- TAGS
        CASE 
            WHEN f.ambiente_servico_tag IS NOT NULL 
                 AND trim(f.ambiente_servico_tag) <> '' 
            THEN TRUE ELSE FALSE 
        END AS tem_tags,

        CASE 
            WHEN f.ambiente_servico_tag IS NOT NULL 
                 AND trim(f.ambiente_servico_tag) <> '' 
            THEN 1 ELSE 0 
        END AS qtd_tags,

        f.ambiente_servico_tag AS tags_raw,

        -- MARCADOR
        CASE 
            WHEN f.marker IS NOT NULL AND trim(f.marker) <> '' 
            THEN TRUE ELSE FALSE 
        END AS foto_tem_marcador,

        -- FOTO CLIENTE
        CAST(f.cliente AS BOOLEAN) AS foto_cliente

    FROM workspace.default.silver_fotos f

    -- 🔒 FILTROS DE QUALIDADE (somente para manter FKs consistentes)
    WHERE f.creator IS NOT NULL AND trim(f.creator) <> ''
      AND f.projeto IS NOT NULL AND trim(f.projeto) <> ''
      AND f.visita IS NOT NULL AND trim(f.visita) <> ''
)

SELECT
    ROW_NUMBER() OVER (ORDER BY b.creation_dt, b.foto_id) AS id_fato_foto,

    -- FKS
    p.projeto_id,
    u.usuario_id,
    v.visita_id,
    t.tempo_id,

    -- ATRIBUTOS
    b.tem_tags,
    b.qtd_tags,
    b.tags_raw,
    b.foto_tem_marcador,
    b.foto_cliente,

    -- DATAS
    b.creation_dt AS date_foto,
    b.creation_dt AS date_foto_mod

FROM base b

LEFT JOIN gold.dim_projeto p
    ON b.projeto = p.nome_projeto

LEFT JOIN gold.dim_usuario u
    ON b.creator = u.email

LEFT JOIN gold.dim_visita v
    ON b.visita = v.nome_visita

LEFT JOIN gold.dim_tempo t
    ON b.creation_dt = t.data;


In [0]:
%sql
SELECT COUNT(*) FROM gold.fato_foto_adicionada;


In [0]:
%sql
SELECT * FROM gold.fato_foto_adicionada limit 20;
